In [1]:
import polars as pl 
import gzip 
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 
import pyarrow.parquet as pq
import pyarrow as pa
import traceback


In [2]:
#parquet_schema

schema_parquet = pa.schema([
    ('Id_Fichier', pa.string()), #int64
    ('IdColl', pa.string()),
    ('DteStr', pa.string()), #string
    ('LibellePoste', pa.string()),
    ('IdPost', pa.string()),
    ('TypOpBudg', pa.string()), #des 2 et des 1
    ('OpeCpteTiers', pa.string()),
    ('NatCEPL', pa.string()),
    ('Departement', pa.string()), #On oublie pas les 2A,
    ('NatDec', pa.string()), #A terme, sera un pa.dicitonnary
    ('Exer', pa.float32()), #int16
    ('ProjetBudget', pa.string()), #bool mais à verif 
    ('NatVote', pa.string()), #dico aussi
    ('OpeEquip', pa.string()), #necessite recherche
    ('VoteFormelChap', pa.string()), #bool mais à verif 
    ('TypProv', pa.string()), #int ? 
    ('BudgPrec', pa.string()), #int ?
    ('ReprRes', pa.string()), #int ?
    ('NatFonc', pa.string()), #int ?
    ('CodTypBud', pa.string()),
    ('LibelleEtabPal', pa.string()),
    ('IdEtabPal', pa.string()),
    ('LibelleEtab', pa.string()),
    ('IdEtab', pa.string()),
    ('CodColl', pa.string()),
    ('CodInseeColl', pa.string()),
    ('CodBud', pa.string()),
    ('Nomenclature', pa.string()),
    ('Nature', pa.string()),
    ('Fonction', pa.string()),
    ('LibCpte', pa.string()),
    ('ContNat', pa.string()),
    ('ArtSpe', pa.string()), #Doit être corrigé en bool,
    ('ContFon', pa.string()),
    ('CodRD', pa.string()), #Val D ou R,
    ('MtBudgPrec', pa.float32()), #float32 #nullable = True
    ('MtRARPrec', pa.float32()), #float32
    ('MtPropNouv', pa.float32()), #float32
    ('MtPrev', pa.float32()), #float32
    ('CredOuv', pa.float32()), #int32
    ('MtReal', pa.float32()), #float32
    ('MtRAR3112', pa.float32()), #float32
    ('OpBudg', pa.string()),
    ('MtSup', pa.string()),
    ('CaracSup', pa.string()),
        ('BudgetHorsRAR', pa.string()),
        ('ICNE', pa.string()),
        ('ICNEPrec', pa.string()),
        ('APVote', pa.string()),
        ('ProdChaRat', pa.string()),
        ('RARPrec', pa.string()),
        ('ProgAutoLib', pa.string()),
        ('ProgAutoNum', pa.string()),
        ('MtOpeCumul', pa.string()),
        ('Brut', pa.string()),
        ('Comp', pa.string()),
        ('Net', pa.string()),
        ('ChapSpe', pa.string()),
        ('MtOpeInfo', pa.string()),
        ('TypOpe', pa.string()),
        ('MtSup_8_Lib', pa.string()),
        ('MtSup_8_Val', pa.string()),
        ('MtSup_9_Lib', pa.string()),
        ('MtSup_9_Val', pa.string()),
        ('MtSup_10_Lib', pa.string()),
        ('MtSup_10_Val', pa.string()), #10
        ('CaracSup_1_Lib', pa.string()),
        ('CaracSup_1_Val', pa.string()),
        ('CaracSup_2_Lib', pa.string()),
        ('CaracSup_2_Val', pa.string()),
        ('CaracSup_3_Lib', pa.string()),
        ('CaracSup_3_Val', pa.string()),
        ('CaracSup_4_Lib', pa.string()),
        ('CaracSup_4_Val', pa.string()),
        ('CaracSup_5_Lib', pa.string()),
        ('CaracSup_5_Val', pa.string()),
        ('CaracSup_6_Lib', pa.string()),
        ('CaracSup_6_Val', pa.string()),
        ('CaracSup_7_Lib', pa.string()), #7
        ('CaracSup_7_Val', pa.string()),
        
    ])

In [6]:
#Fonctions de base

def _recherche_id_dans_parquet(chemin_parquet_db):
 
 conduck = duckdb.connect(database=':memory:', read_only=False)
 source_duck = conduck.read_parquet(f'{chemin_parquet_db}')
 requete = conduck.execute(''' 
    SELECT DISTINCT Id_Fichier 
    FROM source_duck''')
 ligne_sql_int = [int(x[0]) for x in requete.fetchall()]
 conduck.close()
 return ligne_sql_int

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 ''' Pas encore utilisée, permet de récupérer les ID dans le parquet contenant les 
 données déjà traitées '''
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id

def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier

def nettoyage_V(dataframe : pd.DataFrame) -> pd.DataFrame :
 ''' Permet de supprimer les @V des colonnes à l'exception de MtSup et CaracSup'''

 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['MtSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre


In [7]:
def travail_mtsup_ligne(ligne_budg) :
 type_m = ligne_budg.get('MtSup')
 if type_m is None : 
  pass 
 else : 
  if isinstance(type_m, dict) : 
   ligne_budg[type_m.get('@Code', '')] = {'@V' : type_m.get('@V', '')}
  elif isinstance(type_m, list) :
    for z in type_m : #z c'est un dict contenant un V et un Code 
      ligne_budg[z.get('@Code', '')] = {'@V' : z.get('@V', '')}
 return ligne_budg

def travail_caracsup_ligne(ligne_budg) : 
 type_c = ligne_budg.get('CaracSup')
 if type_c is None : 
  pass 
 else : 
  if isinstance(type_c, dict) : 
   ligne_budg[type_c.get('@Code', '')] = {'@V' : type_c.get('@V', '')}
  elif isinstance(type_c, list) : 
   for ii in type_c : 
    ligne_budg[ii.get('@Code', '')] = {'@V' : ii.get('@V', '')}
 return ligne_budg

In [8]:
def task_1_v3(chemin_xml_entree, chemin_parquet_verif, chemin_sortie) : 
 ''' La V2 traite en plus :
  - F Le cas où il n'y a qu'une seule ligne budget (pour caracsup et mtsup).
  - F Rentre au propre MtSup Et CaracSup
  - F Ajoute des chemins que l'on peut choisir
  - F Ajoute une verification depuis un plus grand parquet, ça pourra se faire via une db plus tôt, quand on en aura une
  - F Sert un café robusta pas fou mais buvable
  - F Ajoute de nouvelles métadonnées, dont l'année d'exercice, la nature de vote, le siret d'etab parfois different du premier siret etc. 
  - En cours, Fait le schema à cette étape pour éviter tout soucis après 
 '''

 chemin_xml_entree_glob = glob.glob(os.path.join(chemin_xml_entree, "*.gz"))
 liste_verif_parquet = _recherche_id_dans_parquet(chemin_parquet_db= chemin_parquet_verif)
 liste_fichiers_safe = [chemin_fichier for chemin_fichier in chemin_xml_entree_glob if int(_isolement_id(chemin_fichier)) not in liste_verif_parquet]
 for fichier in liste_fichiers_safe : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['BlocBudget'])
   metadonnees.update(dico['DocumentBudgetaire']['Budget']['EnTeteBudget'])
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   if isinstance(docbase, list):
    for i in docbase : 
      i.update(metadonnees)
      i = travail_mtsup_ligne(i)
      i = travail_caracsup_ligne(i)
   elif isinstance(docbase, dict) : 
    docbase.update(metadonnees)
    docbase = travail_mtsup_ligne(docbase)
    docbase = travail_caracsup_ligne(docbase)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_V(df_base)
   df_propre.to_parquet(f'{chemin_sortie}{id_fichier}_parquet', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(f'Erreur complète : {repr(e)}')
    traceback.print_exc()
    continue 

Test complet sur 20k fichiers 

In [9]:
task_1_v3('.././fichiers/todo_xml/', '.././fichiers/parquet_vide_verif', '.././fichiers/todo_parquet/')

Erreur fichier .././fichiers/todo_xml\775775.xml.gz, extraction impossible
Erreur complète : ExpatError('no element found: line 1, column 0')


Traceback (most recent call last):
  File "C:\Users\v_enz\AppData\Local\Temp\ipykernel_21072\320273135.py", line 18, in task_1_v3
    dico = parse_fichier(fichier)
           ^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\v_enz\AppData\Local\Temp\ipykernel_21072\1877147927.py", line 19, in parse_fichier
    fichier_dict = xmltodict.parse(fichier_xml)
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\python\Lib\site-packages\xmltodict.py", line 378, in parse
    parser.Parse(xml_input, True)
xml.parsers.expat.ExpatError: no element found: line 1, column 0


In [4]:
def task_2(dossier_entree, chemin_parquet_global, schema_cust) :
 liste_parquet_t = []
 for parquet in glob.glob(os.path.join(dossier_entree, "*")) : 
  liste_parquet_t.append(parquet)
 with pq.ParquetWriter(f"{chemin_parquet_global}.parquet", schema=schema_cust) as writer:
    for file in liste_parquet_t:
        table = pq.read_table(file, schema=schema_cust)
        writer.write_table(table)

In [5]:
task_2('.././fichiers/todo_parquet/', './parquet_v3', schema_cust=schema_parquet)

In [8]:

conduck = duckdb.connect(database=':memory:', read_only=False)
source_duck = conduck.read_parquet('./parquet_v3.parquet')

requete_1 = ''' 
    SELECT DISTINCT Exer
    FROM source_duck
    '''
df_result1 = conduck.execute(requete_1).fetchdf()
#conduck.close()
print(df_result1.shape)
df_result1.head(5)

(6, 1)


,Exer
0,2021.0
1,2020.0
2,NaN
3,2022.0
4,2018.0


In [16]:
requete_exer_nan = '''
    SELECT DISTINCT Id_Fichier, Exer
    FROM source_duck
    WHERE Exer NOT IN ('2021.0','2020.0', '2022.0', '2018.0', '2019.0')
'''
df__exer_nan = conduck.execute(requete_exer_nan).fetchdf()
#conduck.close()
print(df__exer_nan.shape)
df__exer_nan.head(5)

(1, 2)


,Id_Fichier,Exer
0,nan,NaN
